In [0]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3"
# !git clone "https://85869b109f25ac5241470005fcd8ead1673b1329@github.com/rohanrajpal/dlassignment1.git"

!git config --global user.email "rohan17089@iiitd.ac.in"
!git config --global user.name "Rohan Rajpal"

%load_ext autoreload


# Imports


In [0]:
import argparse
import collections

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval
from matplotlib import pyplot as plt
import csv, json
import os
import pandas as pd
from numpy.random import RandomState

In [0]:
PATH_TO_DATASET = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data"
PATH_TO_WEIGHTS = PATH_TO_DATASET + "/pretrained_weights_cleaned.pt"
ANNOTATION_CSV ="/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv"
CLASS_LIST ="/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/class_list.csv"

IMAGE_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images/train"
INSTANCES_TRAIN_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train.json"
INSTANCES_TRAIN_ALT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_alt.json"

TRAIN_CSV = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/train.csv"
VAL_CSV = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/val.csv"
PATH_TO_FINETUNED_WEIGHTS = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/model_final.pt"
BEST_WEIGHTS = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/csv_retinanet_best_model.pt"

data_dir = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/data/"

annotations_json_path = data_dir + "train_annotations.json"
annotations_csv_path = data_dir + "train_annotations_csv.csv"

val_csv = annotations_csv_path
train_csv = data_dir + "train.csv"


weights_path = data_dir + "pretrained_weights.pt"
images_path = data_dir + "train"
# os.system("unzip -q \"{}\" -d \"{}\"".format(images_path,data_dir))

best_model_path = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/third/csv_retinanet_best_model_mAP.pt"
class_list = data_dir + "class_list.csv"

# Preprocessing


In [0]:
# Clean the json
with open(annotations_json_path) as f:
  annot_json = json.load(f)

annot_json_cleaned = {}

annot_json_cleaned["info"] = annot_json["info"]
annot_json_cleaned["categories"] = annot_json["categories"]
annot_json_cleaned["images"] = []
annot_json_cleaned["annotations"] = []

file_path = {}
for path, subdirs, files in os.walk(images_path):
    if files:
        # print(os.path.join(path, min(files)))
        for file in files:
          # print(file)
          file_path[file] = os.path.join(path, file)

def inFolder(elem):
  if elem in file_path:
    return True
  return False

# remove images which arent there
for elem in annot_json["images"]:
  if inFolder(elem["file_name"]):
    annot_json_cleaned["images"].append(elem)

for elem in annot_json["annotations"]:
  if inFolder(elem["image_id"]+".jpg"):
    annot_json_cleaned["annotations"].append(elem)

annot_json = annot_json_cleaned

print("JSON cleaned")

# Generate class list
f_csv = open(class_list, 'w', newline='')

writer = csv.writer(f_csv)
lines = [
["bird","0"],
["bobcat","1"],
["car","2"],
["cat","3"],
["raccoon","4"],
["rabbit","5"],
["coyote","6"],
["squirrel","7"]
]
for line in lines:
  writer.writerow(line)

print("Made class list")
# Generate annotation CSV
class_map = {
11:"bird",
6:"bobcat",
33:"car",
16:"cat",
3:"raccoon",
10:"rabbit",
9:"coyote",
5:"squirrel"
}

f_csv = open(annotations_csv_path, 'w', newline='')
writer = csv.writer(f_csv)

for anno in annot_json["annotations"]:
  bbox = [int(x) for x in anno["bbox"]]
  bbox[2] = bbox[0] + bbox[2]
  bbox[3] = bbox[1] + bbox[3]
  if(anno["category_id"] in class_map):
    class_name = class_map[anno["category_id"]]
    img_path = images_path +"/"+class_name+"/" +anno["image_id"] + ".jpg"
    # print(img_path)
    writer.writerow([img_path,bbox[0],bbox[1],bbox[2],bbox[3],class_name])

print("Made annot CSV")

# Train-Val split
df = pd.read_csv(annotations_csv_path,header=None)
rng = RandomState()

train = df.sample(frac=0.7, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

# Make sure you do a good shuffle in csv
train = train.iloc[np.random.permutation(len(train))]
test = test.iloc[np.random.permutation(len(test))]

train.to_csv(train_csv,index=False,header=None)
test.to_csv(val_csv,index=False,header=None)


# Part 1 : Without finetuning

In [0]:
# Load val dataset
dataset_val = CSVDataset(train_file = val_csv, class_list=class_list,
                                    transform=transforms.Compose([Normalizer(),Resizer()]))
print("Loaded val dataset")
# Eval karo
use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()
retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = False
retinanet.eval()
retinanet.module.freeze_bn()

mAP = csv_eval.evaluate(dataset_val, retinanet)

def avg(mAP):
  avg_map = 0
  for key in mAP:
    avg_map += mAP[key][0]
  avg_map /= 8

  print("Avg mAP",avg_map)

  return avg_map
print(avg(mAP))
print(mAP)

# Part 2 : With Finetuning and Loss Plots

In [0]:
%%time
BEST_WEIGHTS = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/csv_retinanet_best_model.pt"

parser = {"csv_classes":CLASS_LIST, "csv_val":VAL_CSV, "csv_train":TRAIN_CSV, "epochs":10 }

dataset_train = CSVDataset(train_file=parser["csv_train"], class_list=parser["csv_classes"],
                                   transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
dataset_val = CSVDataset(train_file=parser["csv_val"], class_list=parser["csv_classes"],
                                     transform=transforms.Compose([Normalizer(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_train, batch_size=2, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)

retinanet = model.resnet50(num_classes=dataset_train.num_classes(),)

# for param in retinanet.parameters():
#     param.requires_grad = False
# for param in retinanet.classificationModel.output.parameters():
#     param.requires_grad = True
# for param in retinanet.regressionModel.output.parameters():
#     param.requires_grad = True

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

PATH_TO_DATASET = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data"
PATH_TO_WEIGHTS = PATH_TO_DATASET + "/pretrained_weights_cleaned.pt"

retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))
# retinanet = torch.load(BEST_WEIGHTS)

retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, retinanet.parameters()), lr=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

loss_hist = collections.deque(maxlen=500)

loss_all_epochs = []
map_all_epochs = []

def avg(mAP):
  avg_map = 0
  for key in mAP:
    avg_map += mAP[key][0]
  avg_map /= 8

  print("Avg mAP",avg_map)

  return avg_map


retinanet.train()
retinanet.module.freeze_bn()
file1 = open("logfreezefour.txt","w")
print('Num training images: {}'.format(len(dataset_train)))

best_loss = 100
best_map = 0
for epoch_num in range(parser["epochs"]):
    print("\nEpoch",epoch_num)
    retinanet.train()
    retinanet.module.freeze_bn()

    # if epoch_num == 10:
    #   for param in retinanet.parameters():
    #     param.requires_grad = True
    #   optimizer = optim.Adam(filter(lambda p: p.requires_grad, retinanet.parameters()), lr=1e-5)
    
    # if epoch_num == 10:
    #   for param in retinanet.parameters():
    #       param.requires_grad = False
    #   for param in retinanet.classificationModel.output.parameters():
    #       param.requires_grad = True
    #   for param in retinanet.regressionModel.output.parameters():
    #       param.requires_grad = True
    #   optimizer = optim.Adam(filter(lambda p: p.requires_grad, retinanet.parameters()), lr=1e-5)

    epoch_loss = []

    for iter_num, data in enumerate(dataloader_train):
        try:
            optimizer.zero_grad()

            classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()

            loss = classification_loss + regression_loss

            if bool(loss == 0):
                continue

            loss.backward()

            torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)

            optimizer.step()

            loss_hist.append(float(loss))

            epoch_loss.append(float(loss))

            # print(
            #     'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
            #         epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
            
            file1.write('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f} \n'.format(
                    epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
            
            del classification_loss
            del regression_loss
        except Exception as e:
            print(e)
            continue

    print('Evaluating dataset')

    mAP = csv_eval.evaluate(dataset_val, retinanet)
    avg_mAP = avg(mAP)
    map_all_epochs.append(avg_mAP)

    mean_loss = np.mean(epoch_loss)

    scheduler.step(mean_loss)

    loss_all_epochs.append(mean_loss)


    if(mean_loss < best_loss):
        torch.save(retinanet.module, 'freeze_model/fourth/{}_retinanet_{}.pt'.format("csv", "best_model_loss"))
        best_loss = mean_loss
    
    if(avg_mAP > best_map):
      torch.save(retinanet.module, 'freeze_model/fourth/{}_retinanet_{}.pt'.format("csv", "best_model_mAP"))
      best_map = avg_mAP
    
    torch.save(retinanet.module, 'freeze_model/fourth/{}_retinanet_{}.pt'.format("csv", epoch_num))
    print("Best loss:",best_loss,"Best mAP:",best_map)

retinanet.eval()
file1.close()
torch.save(retinanet, 'freeze_model/fourth/model_final.pt')

print("Loss curve")
x_axis = [x for x in range(parser["epochs"])]
plt.plot(x_axis,loss_all_epochs)
plt.xlabel("Epochs")
plt.ylabel("Running Loss")
plt.savefig("/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/fourth/losscurve.png")

plt.figure()
plt.xlabel("Epochs")
plt.ylabel("mAP")
plt.plot(x_axis,map_all_epochs)
plt.savefig("/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/fourth/mapcurve.png")

# Confusion Matrix

In [0]:
def draw_caption(image, box, caption):
  b = np.array(box).astype(int)
  cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
  cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)


In [0]:
parser = {"csv_classes":CLASS_LIST, "csv_val":VAL_CSV, "model_finetuned":BEST_WEIGHTS, "model_raw": PATH_TO_WEIGHTS}
dataset_val = CSVDataset(train_file=parser["csv_val"], class_list=parser["csv_classes"], transform=transforms.Compose([Normalizer(), Resizer()]))

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

retinanet_raw = model.resnet50(num_classes=8)

retinanet_raw.load_state_dict(torch.load(parser["model_raw"]))
retinanet_finetuned = torch.load(parser["model_finetuned"])
torch.save(retinanet_finetuned.state_dict(), TO_SAVE_WT)


def get_bbox(retinanet,img,bcolor,data,caption):
  scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
  # print("--------Classification-----------")
  # print(classification.shape,classification)
  # print("--------Scores----------")
  # print(scores.shape,scores)
  # print("---------Transformed_Idxs-------")
  # print(transformed_anchors.shape, transformed_anchors)
  idxs = np.where(scores.cpu()>0.5)
  # print(idxs[0])
  # bbox = transformed_anchors[idxs[0][j], :]
  class_labels = []
  for j in range(idxs[0].shape[0]):
      bbox = transformed_anchors[idxs[0][j], :]
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(classification[idxs[0][j]])]
      draw_caption(img, (x1, y1, x2, y2), label_name)
      class_labels.append(label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=bcolor, thickness=2)
      # print(caption,label_name)
  return class_labels


def equalize(pred, truth):
  diff = len(pred) - len(truth)
  nothing_pad = ["nothing" for x in range(abs(diff))]
  if(diff > 0):
    truth += nothing_pad
  elif(diff < 0):
    pred += nothing_pad
  return pred, truth
  
use_gpu = True

if use_gpu:
  retinanet_raw = retinanet_raw.cuda()
  retinanet_finetuned = retinanet_finetuned.cuda()

retinanet_raw.eval()
retinanet_finetuned.eval()

unnormalize = UnNormalizer()

# fig, ax = plt.subplots(figsize=(20, 10))
classcnt = [3 for x in range(8)]
sum = 24


raw_pred_list = []
finetune_pred_list = []
gt_raw_list = []
gt_finetune_list = []
cnt = 1 
for idx, data in enumerate(dataloader_val):
  # print(int(data['annot'][0][0][4]))

  # 3 images for each class
  # category = int(data['annot'][0][0][4])
  # if(classcnt[category] > 0):
  #   classcnt[category] -= 1
  #   sum -= 1
  # elif sum <= 0:
  #   break
  # else:
  #   continue
  
  with torch.no_grad():
    
    img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

    img[img<0] = 0
    img[img>255] = 255

    img = np.transpose(img, (1, 2, 0))

    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

    # st = time.time()
    raw_pred = get_bbox(retinanet_raw,img,(0,0,255),data,str(cnt)+" Model:")
    finetune_pred = get_bbox(retinanet_finetuned,img,(0,255,0),data,str(cnt)+" Finetuned model:")
    gt_raw = []
    gt_finetune = []
    # Ground truth
    # print(data['annot'])

    for bbox in data['annot'][0]:
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(bbox[4])]
      gt_raw.append(label_name)
      gt_finetune.append(label_name)
      draw_caption(img, (x1, y1, x2, y2), label_name)
      # print(str(cnt)+" Ground truth:",label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=(255,0,0), thickness=2)
      # print(label_name)

    raw_pred,gt_raw = equalize(raw_pred,gt_raw)
    finetune_pred,gt_finetune = equalize(finetune_pred,gt_finetune)

    # print("Raw pretrained",raw_pred,"Ground",gt_raw)
    # print("Finetune pred",finetune_pred,"Ground",gt_finetune)

    raw_pred_list += raw_pred
    finetune_pred_list += finetune_pred

    gt_raw_list += gt_raw
    gt_finetune_list += gt_finetune

    # print("Ground truth",gt)
    # print("------------------------")
    # print('Elapsed time: {}'.format(time.time()-st))

    # plt.figure(figsize=(20,10)) 
    # plt.imshow(img)
    # plt.savefig("images1/"+str(cnt)+"classification.png")
    # print("---------------------------------")
    cnt += 1
    # plt.show()

# print(len(raw_pred_list),len(gt_raw_list))
# print(len(finetune_pred_list),len(gt_finetune_list))

# print((raw_pred_list),(gt_raw_list))
# print((finetune_pred_list),(gt_finetune_list))

print(confusion_matrix(gt_raw_list,raw_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))
print(confusion_matrix(gt_finetune_list,finetune_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))

In [0]:
parser = {"csv_classes":CLASS_LIST, "csv_val":VAL_CSV, "model_finetuned":BEST_WEIGHTS, "model_raw": PATH_TO_WEIGHTS}
dataset_val = CSVDataset(train_file=parser["csv_val"], class_list=parser["csv_classes"], transform=transforms.Compose([Normalizer(), Resizer()]))

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

retinanet_raw = model.resnet50(num_classes=8)

retinanet_raw.load_state_dict(torch.load(parser["model_raw"]))
retinanet_finetuned = torch.load(parser["model_finetuned"])
torch.save(retinanet_finetuned.state_dict(), TO_SAVE_WT)


def get_bbox(retinanet,img,bcolor,data,caption):
  scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
  # print("--------Classification-----------")
  # print(classification.shape,classification)
  # print("--------Scores----------")
  # print(scores.shape,scores)
  # print("---------Transformed_Idxs-------")
  # print(transformed_anchors.shape, transformed_anchors)
  idxs = np.where(scores.cpu()>0.5)
  # print(idxs[0])
  # bbox = transformed_anchors[idxs[0][j], :]
  class_labels = []
  for j in range(idxs[0].shape[0]):
      bbox = transformed_anchors[idxs[0][j], :]
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(classification[idxs[0][j]])]
      draw_caption(img, (x1, y1, x2, y2), label_name)
      class_labels.append(label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=bcolor, thickness=2)
      # print(caption,label_name)
  return class_labels


def equalize(pred, truth):
  diff = len(pred) - len(truth)
  nothing_pad = ["nothing" for x in range(abs(diff))]
  if(diff > 0):
    truth += nothing_pad
  elif(diff < 0):
    pred += nothing_pad
  return pred, truth
  
use_gpu = True

if use_gpu:
  retinanet_raw = retinanet_raw.cuda()
  retinanet_finetuned = retinanet_finetuned.cuda()

retinanet_raw.eval()
retinanet_finetuned.eval()

unnormalize = UnNormalizer()

# fig, ax = plt.subplots(figsize=(20, 10))
classcnt = [3 for x in range(8)]
sum = 24


raw_pred_list = []
finetune_pred_list = []
gt_raw_list = []
gt_finetune_list = []
cnt = 1 
for idx, data in enumerate(dataloader_val):
  # print(int(data['annot'][0][0][4]))

  # 3 images for each class
  category = int(data['annot'][0][0][4])
  if(classcnt[category] > 0):
    classcnt[category] -= 1
    sum -= 1
  elif sum <= 0:
    break
  else:
    continue
  
  with torch.no_grad():
    
    img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

    img[img<0] = 0
    img[img>255] = 255

    img = np.transpose(img, (1, 2, 0))

    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

    # st = time.time()
    raw_pred = get_bbox(retinanet_raw,img,(0,0,255),data,str(cnt)+" Model:")
    finetune_pred = get_bbox(retinanet_finetuned,img,(0,255,0),data,str(cnt)+" Finetuned model:")
    gt_raw = []
    gt_finetune = []
    # Ground truth
    # print(data['annot'])

    for bbox in data['annot'][0]:
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(bbox[4])]
      gt_raw.append(label_name)
      gt_finetune.append(label_name)
      draw_caption(img, (x1, y1, x2, y2), label_name)
      # print(str(cnt)+" Ground truth:",label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=(255,0,0), thickness=2)
      # print(label_name)

    raw_pred,gt_raw = equalize(raw_pred,gt_raw)
    finetune_pred,gt_finetune = equalize(finetune_pred,gt_finetune)

    # print("Raw pretrained",raw_pred,"Ground",gt_raw)
    # print("Finetune pred",finetune_pred,"Ground",gt_finetune)

    raw_pred_list += raw_pred
    finetune_pred_list += finetune_pred

    gt_raw_list += gt_raw
    gt_finetune_list += gt_finetune

    # print("Ground truth",gt)
    # print("------------------------")
    # print('Elapsed time: {}'.format(time.time()-st))

    # plt.figure(figsize=(20,10)) 
    plt.imshow(img)
    # plt.savefig("images1/"+str(cnt)+"classification.png")
    print("---------------------------------")
    cnt += 1
    plt.show()

# print(len(raw_pred_list),len(gt_raw_list))
# print(len(finetune_pred_list),len(gt_finetune_list))

# print((raw_pred_list),(gt_raw_list))
# print((finetune_pred_list),(gt_finetune_list))

print(confusion_matrix(gt_raw_list,raw_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))
print(confusion_matrix(gt_finetune_list,finetune_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))